Take top 10% from first Generation

With crossover create new 80% from my 10%

The rest 20% are totally random

Repeat for each iteration!

Checking the new Methodology on Iris Dataset

In [43]:
# Import TensorFlow and other libraries
from concurrent.futures import ProcessPoolExecutor
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential, clone_model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import numpy as np
import time
from joblib import Parallel, delayed


# Import the Iris data set from sklearn
from sklearn.datasets import load_iris

# Define the output size
output_size = 3 # change this according to your problem

print("define all functions..")

# Define the custom loss function
##### OLD VERSION #####
# def custom_loss(y_true, y_pred):
#     """ 
#     y_ture [batch_size x output_shape]
    
#     return [batch_size x 1]
#     """
#     y_true_ind = tf.map_fn(fn = lambda t: tf.argmax(t), elems=y_true, dtype=tf.int64)
#     y_pred_ind = tf.map_fn(fn = lambda t: tf.argmax(t), elems=y_pred, dtype=tf.int64)
#     total_loss = tf.map_fn(fn = lambda t: tf.cast(t[0] == t[1], tf.float32), elems=(y_true_ind, y_pred_ind), dtype=tf.float32)
#     return total_loss

def custom_loss(y_true, y_pred):
    y_true_ind = tf.argmax(y_true, axis=1)
    y_pred_ind = tf.argmax(y_pred, axis=1)
    total_loss = tf.cast(tf.equal(y_true_ind, y_pred_ind), tf.float32)
    return total_loss

# Define the fitness function
def fitness(model, x, y, batch_size):
    # Evaluate the model on the data and return the accuracy
    # t = time.time()
    y_pred = model.predict(x, batch_size=batch_size, verbose=0)
    # tt = time.time()
    accuracy = tf.reduce_mean(custom_loss(y, y_pred)).numpy()
    # ttt = time.time()
    
    # print("Predict: {:.2f}s, Loss: {:.2f}s".format(tt - t, ttt - tt))
    return accuracy

# Define the mutation function
def mutate(weights, mutation_rate):
    # Apply random changes to the weights with a given probability
    new_weights = []
    for w in weights:
        shape = w.shape
        w = w.flatten()
        for i in range(len(w)):
            if np.random.random() < mutation_rate:
                w[i] += np.random.normal(0, 0.3) # add a small random noise  95%  -0.9 -- 0.9
        w = w.reshape(shape)
        new_weights.append(w)
    return new_weights

# Define the crossover function
def crossover(weights1, weights2, crossover_rate):
    # Combine the weights of two parents with a given probability
    new_weights1 = []
    new_weights2 = []
    for w1, w2 in zip(weights1, weights2):
        shape = w1.shape
        w1 = w1.flatten()
        w2 = w2.flatten()
        if np.random.random() < crossover_rate:
            
            ### Probablly this piece of code takes too long to run
            # # Choose a random crossover point
            # point = np.random.randint(len(w1))
            # # Swap the weights after the crossover point
            # w1[point:], w2[point:] = w2[point:], w1[point:]
            
            w1, w2 = w2, w1
            
        w1 = w1.reshape(shape)
        w2 = w2.reshape(shape)
        new_weights1.append(w1)
        new_weights2.append(w2)
    return new_weights1, new_weights2

# Define the selection function
def select(population, fitnesses, elitism_rate):
    # Sort the population by fitness in descending order
    sorted_indices = np.argsort(fitnesses)[::-1]
    population = [population[i] for i in sorted_indices]
    fitnesses = [fitnesses[i] for i in sorted_indices]

    # Keep the best individuals as elites
    n_elites = int(elitism_rate * len(population))
    elites = population[:n_elites]

    # Select the rest of the individuals by roulette wheel method
    selected = []
    total_fitness = np.sum(fitnesses[n_elites:])
    probabilities = [f / total_fitness for f in fitnesses[n_elites:]]
    for _ in range(len(population) - n_elites):
        r = np.random.random()
        s = 0
        for i in range(n_elites, len(population)):
            s += probabilities[i - n_elites]
            if s >= r:
                selected.append(population[i])
                break
    
    # Return the elites and the selected individuals
    return elites + selected

# Define the build and compile model function
def build_and_compile_model():
    model = Sequential([
        Input(shape=(4,)),
        Dense(2*output_size, activation="relu"),
        Dense(8*output_size, activation="relu"),
        Dense(32*output_size, activation="relu"),
        Dense(output_size)
    ])

    model.compile(loss=custom_loss,
                  optimizer=Adam(0.001))
    
    model.build(input_shape=(3,))
    
    return model

define all functions..


In [44]:
# Define the genetic algorithm parameters
population_size = 100 # number of individuals in the population
mutation_rate = 0.1 # probability of mutation for each gene
crossover_rate = 0.8 # probability of crossover for each pair of parents
elitism_rate = 0.1 # fraction of the best individuals to keep in each generation
max_generations = 100 # maximum number of generations to run the algorithm
batch_size = 32

In [45]:
import time

start_time = time.time()
start_time

1683829691.3833776

In [46]:
x.shape, y.shape

((150, 4), TensorShape([150, 3]))

In [47]:
print("Prepare data...")

# Load the data and the labels
data = load_iris()
x = data.data # shape = (150, 4)
y = data.target # shape = (150,)
y = tf.one_hot(y, depth=output_size) # convert to one-hot encoding

print("Initialise models...")

# Initialize the population with random models
population = []
for _ in range(population_size):
    model = build_and_compile_model()
    population.append(model)

print("Start learning...")

# Run the genetic algorithm for a given number of generations
for generation in range(max_generations):
    
    start_time = time.time()
    
    # Evaluate the fitness of each individual
    fitnesses = []
    for model in population:
        fitnesses.append(fitness(model, x, y, batch_size))
        
    fit_finish_time = time.time()
    
    # Print the average and best fitness in the current generation
    avg_fitness = np.mean(fitnesses)
    best_fitness = np.max(fitnesses)
    print(f"Generation {generation}: Average fitness = {avg_fitness}, Best fitness = {best_fitness}")

    # Check if the best fitness is 1.0 (perfect accuracy)
    if best_fitness == 1.0:
        print("Found the optimal solution!")
        break

    # Select the individuals for the next generation
    population = select(population, fitnesses, elitism_rate)
    
    population_select_time = time.time()

    # Apply crossover and mutation to generate new individuals
    new_population = []
    while len(new_population) < population_size - 20:
        # Choose two parents randomly
        parent1 = np.random.choice(population)
        parent2 = np.random.choice(population)

        # Clone parents' models
        child1 = clone_model(parent1)
        child2 = clone_model(parent2)

        # Get parents' weights
        weights1 = parent1.get_weights()
        weights2 = parent2.get_weights()

        # Apply crossover
        child_weights1, child_weights2 = crossover(weights1, weights2, crossover_rate)

        # Apply mutation
        child_weights1 = mutate(child_weights1, mutation_rate)
        child_weights2 = mutate(child_weights2, mutation_rate)

        # Set new weights for child models
        child1.set_weights(child_weights1)
        child2.set_weights(child_weights2)

        # Add children to the new population
        new_population.append(child1)
        new_population.append(child2)
        
    new_population_time = time.time()
    
    print("Time for fit: {:.2f}s, Selection: {:.2f}s, new_population_time: {:.2f}s".format(fit_finish_time-start_time,
                                                                                           population_select_time-fit_finish_time,
                                                                                           new_population_time-population_select_time))
        
    # Add 20 completely random models
    for _ in range(20):
        model = build_and_compile_model()
        new_population.append(model)
    
    # Replace the old population with the new one
    population = new_population

# Find the best model in the final population
best_model = None
best_fitness = 0.0
for model in population:
    f = fitness(model, x, y)
    if f > best_fitness:
        best_model = model
        best_fitness = f

# Print the best model and its weights
print("The best model is:")
best_model.summary()
print("The weights are:")
for w in best_model.get_weights():
    print(w)

Prepare data...
Initialise models...
Start learning...
